In [ ]:
from datasets import load_dataset
dataset = load_dataset("beomi/KoAlpaca-v1.1a")

In [1]:
dataset

NameError: name 'dataset' is not defined

In [ ]:
# Assuming dataset is a dictionary with the structure dataset["train"]["output"]
train_outputs = dataset["train"]["output"]

# Find the longest sentence using max with a key function
longest_sentence = max(train_outputs, key=len)

print("The longest sentence is:")
print(longest_sentence)
print("Length of the longest sentence:", len(longest_sentence))


In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)

import torch 
from datasets import load_dataset

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "./llama-3-korean-8b-hf/checkpoint-500",
    attn_implementation="flash_attention_2", # SDPA를 사용하고, 대안으로 'flash_attention_2'를 사용할 수 있음
    torch_dtype=torch.bfloat16,
    use_cache=False,
    device_map="auto"
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:

LLAMA_3_CHAT_TEMPLATE = (
    "{% for message in messages %}"
        "{% if message['role'] == 'system' %}"
            "{{ message['content'] }}"
        "{% elif message['role'] == 'user' %}"
            "{{ '\n\nHuman: ' + message['content'] +  eos_token }}"
        "{% elif message['role'] == 'assistant' %}"
            "{{ '\n\nAssistant: '  + message['content'] +  eos_token  }}"
        "{% endif %}"
    "{% endfor %}"
    "{% if add_generation_prompt %}"
    "{{ '\n\nAssistant: ' }}"
    "{% endif %}"
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("./llama-3-korean-8b-hf", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = LLAMA_3_CHAT_TEMPLATE

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.push_to_hub("daje/meta-llama3-8B-qna-koalpaca-v1.1")
tokenizer.push_to_hub("daje/meta-llama3-8B-qna-koalpaca-v1.1")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

In [9]:
from transformers import pipeline


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    return_full_text=False,
    do_sample=True,
    max_new_tokens=2048,
    temperature=0.7,
)


In [5]:
import os 
test_dataset = load_dataset(
    "json",
    data_files=os.path.join("", "./test_dataset.json"),
    split="train",
)

In [11]:
pipe('케첩과 마요네즈의 입구 모양은 왜 다른가요? \n보통 케찹과 마요네즈는 한 세트자나요? 그런데 보면 케찹은 원터치 뚜껑에다가 새 케찹은 뚜껑을 돌려서 열면 얇은 알미늄 같은 막도 이꾸또 케찹이 나오는 구멍이 동그랗자나요? 근데 마요네즈는 돌리는 뚜껑에다가 처음에 새로 사도 알미늄 같은 막두 엄꾸나오는 구멍 모양도 별 모양인데 왜 그런가요?'
     , max_new_tokens=128
     , do_sample=True
   )

[{'generated_text': ''}]

In [6]:
test_dataset

Dataset({
    features: ['messages'],
    num_rows: 2116
})

In [7]:
test_dataset[0]["messages"][:2]

[{'content': '당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다. 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요.',
  'role': 'system'},
 {'content': '케첩과 마요네즈의 입구 모양은 왜 다른가요? \n보통 케찹과 마요네즈는 한 세트자나요? 그런데 보면 케찹은 원터치 뚜껑에다가 새 케찹은 뚜껑을 돌려서 열면 얇은 알미늄 같은 막도 이꾸또 케찹이 나오는 구멍이 동그랗자나요? 근데 마요네즈는 돌리는 뚜껑에다가 처음에 새로 사도 알미늄 같은 막두 엄꾸나오는 구멍 모양도 별 모양인데 왜 그런가요?',
  'role': 'user'}]

In [8]:
from datasets import load_dataset 
from random import randint


# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))
messages = eval_dataset[rand_idx]["messages"][:2]
print(tokenizer.eos_token)
# stop generation on eos token or <|eot_id|> token
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

# Test on sample 
input_ids = tokenizer.apply_chat_template(messages,add_generation_prompt=True,return_tensors="pt").to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.2,
    top_p=0.95,
)
response = outputs[0][input_ids.shape[-1]:]
print(f"**Query:**\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print(f"**Original Answer:**\n  {eval_dataset[rand_idx]['messages'][2]['content']}")
print(f"**Generated Answer:**\n {tokenizer.decode(response,skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|end_of_text|>
**Query:**
해삼을 먹을 때 먹을 수 없는 해삼이 있는가요? 그러한 경우를 어떻게 구분할 수 있을까요?
**Original Answer:**
  바다에서 스쿠버 다이빙을 하거나 해녀가 수집한 해삼 중 개해삼이라는 종류가 있습니다. 개해삼은 색깔이 고동색으로 전혀 먹을 수 없으며, 목삼이라고도 부릅니다. 목삼은 씹으면 입안이 무거워지며, 이도 찌르는 느낌이 듭니다. 따라서, 개해삼이나 목삼은 먹을 수 없는 해삼입니다. 해삼을 구분하는 방법은 해삼의 색깔과 개체의 모양, 잇빨로 깨물었을 때의 느낌 등으로 가능합니다. 

해삼은 참해삼(홍해삼, 청해삼), 목삼, 그리고 개해삼으로 나뉩니다. 목삼은 보통 해산물 가공 과정에서 건해삼으로 만들어 식재료로 이용됩니다. 건해삼을 만들기 위해서는 배를 가르고 끓임으로써 물을 제거해야합니다. 김이 올라오면 찬물을 한 번 껴 넣어준 뒤 끓여내면 됩니다. 건조와정에서는 습기가 없는 장소에서 태양과 바람에 말려야합니다. 이 과정은 시간이 걸리며, 건조기를 이용하는 것보다 자연에서 건조할 때 더 좋은 효과를 볼 수 있습니다.

참해삼의 경우에는 돌기가 있으며 찌르는 느낌이 들지만, 먹을 수 있는 해삼입니다. 참해삼은 붉은 색의 홍삼과 청록색을 띄는 참해삼이 있으며, 먹는 방법은 다양합니다. 하지만, 개해삼과목삼을 제외하고는 대체로 먹을 수 있는 해삼입니다.
**Generated Answer:**
 1.5~1.1. 
- 1.1. 1.5, 1.5, 2.5, 1.5, 1.5, 2.5, 1.5, 2.5. 3.5. 1.5. 2.5. 1.5, 2.5. 3.5. 4.5. 2.5, 2.5. 2.5, 4.5, 2.5, 1.5, 2.5, 2.5, 3.5. 1.5, 3.5, 1.5, 1.5, 4.5, 2.5, 1, 2.5, 3. 2.5, 4. 2. 2. 2. 2. 1. 1. 4. 1. 3. 3. 1. 3. 4. 4. 4. 1. 3. 1. 4. 4. 4. 2. 2. 2. 2. 3. 4. 4. 4. 4. 2. 1. 5. 

In [11]:
tokenizer.decode(outputs[0])

'당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다. 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요.\n\nHuman: 2007년에 새롭게 선정된 세계 7대 불가사의가 무엇인지 알고싶습니다. 이전에는 어떤 7대 불가사의였나요?<|end_of_text|>\n\nAssistant: 1.1. 1.2, 1.5, 1.2, 2.5, 1.5, 1.1, 1, 2, 2.2, 1, 1, 2, 2.2, 1, 2.5, 1.5, 1.1, 1, 2.2. 1. 1, 2.5, 1. 1, 1, 1, 2.1, 1, 1.2, 1, 1, 1, 1, 1, 1, 1.1, 1, 1, 1, 2. 1, 2, 1, 1, 1, 2.5, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 3, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1,